2.3-Perceptrón

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# CARGA DE CONJUNTO DE DATOS
iris = load_iris()
X = iris.data[:, :2] 
y = iris.target

# PARA SOLO TENER LAS 2 CLASES
filtro = y < 2
X, y = X[filtro], y[filtro]

# División de datos de entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.3, random_state=42)


class PerceptronModificado(object):
    def __init__(self, tasa_de_aprendizaje=0.01, numero_de_iteraciones=50):
        self.tasa_de_aprendizaje = tasa_de_aprendizaje
        self.numero_de_iteraciones = numero_de_iteraciones

    def fit(self, X, y):
        n_muestras, n_caracteristicas = X.shape
        self.pesos = np.zeros(n_caracteristicas + 1)
        self.errores_ = []

        for _ in range(self.numero_de_iteraciones):
            errores = 0
            for i in range(n_muestras):
                xi = X[i]
                objetivo = y[i]
                prediccion = self.predict(xi)
                error = objetivo - prediccion
                self.pesos[1:] += self.tasa_de_aprendizaje * error * xi
                self.pesos[0] += self.tasa_de_aprendizaje * error
                errores += int(error != 0.0)
            self.errores_.append(errores)
        return self

    def predict(self, X):
        z = np.dot(X, self.pesos[1:]) + self.pesos[0]
        return np.where(z >= 0, 1, 0)


perceptron = PerceptronModificado(tasa_de_aprendizaje=0.1, numero_de_iteraciones=10)
perceptron.fit(X_entrenamiento, y_entrenamiento)

# Evaluación de modelo
y_pred = perceptron.predict(X_prueba)
precision = accuracy_score(y_prueba, y_pred)


def visualizar_frontera(X, y, modelo):
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.02),
                           np.arange(x2_min, x2_max, 0.02))
    Z = modelo.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1], alpha=0.8, c=[cmap(idx)], marker=markers[idx], label=cl)

# Frontera de decisión
plt.figure(figsize=(10, 6))
visualizar_frontera(X_prueba, y_prueba, perceptron)
plt.xlabel('Longitud del sépalo')
plt.ylabel('Ancho del sépalo')
plt.title(f'Frontera de Decisión - Perceptrón Modificado (Precisión: {precision*100:.2f}%)')
plt.legend(loc='upper left')
plt.show()


El modelo alcanzó una precisión del 100% en el conjunto de prueba. Esto significa que el modelo fue capaz de clasificar correctamente todas las muestras de prueba, identificando correctamente si pertenecían a la clase 0 o la clase 1 basándose en la longitud y el ancho del sépalo.Se eligió esta metrica de desempeño debido a la simplicidad del problema y el balanceo de las clases.

Este alto nivel de precisión indica  que, para este conjunto de datos particular y las clases seleccionadas, el perceptrón modificado ha aprendido de manera efectiva la frontera de decisión entre las dos clases.

El alto rendimiento del perceptrón en la clasificación binaria del dataset de Iris se atribuye a características claras que diferencian las clases, un balance de clases que valida la precisión como métrica, y la simplicidad del problema, lo cual es óptimo para el perceptrón.

Ejercico 2.2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif

# Cargar el dataset
file_path = "high_diamond_ranked_10min.csv"  # Reemplaza con la ruta correcta a tu archivo
data = pd.read_csv(file_path)

# Eliminar la columna 'gameId'
data = data.drop(columns=['gameId'])

# Separar la variable objetivo y las caracteristicas
X = data.drop('blueWins', axis=1)
y = data['blueWins']

# Escalar las caracter�sticas al rango [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Regresion Logistica
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = logistic_model.predict(X_test)

# Evaluar el modelo original
precision_original = precision_score(y_test, y_pred)
print(f"Original Logistic Regression Model Precision: {precision_original}")

# Definir la m�trica de desempe�o (por ejemplo, precisi�n)
performance_metric = precision_score

# Definir las t�cnicas de selecci�n de caracter�sticas
feature_selection_techniques = [
    ('Univariate Selection (ANOVA F-statistic)', SelectKBest(f_classif, k=5)),
    ('Chi-squared', SelectKBest(chi2, k=5)),
    ('Mutual Information', SelectKBest(mutual_info_classif, k=5))
]

# Ajustar y evaluar el modelo para cada t�cnica de selecci�n de caracter�sticas
for name, selector in feature_selection_techniques:
    # Seleccionar caracter�sticas
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Crear y entrenar el modelo SVM
    svm_model = SVC()
    svm_model.fit(X_train_selected, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred_svm = svm_model.predict(X_test_selected)

    # Evaluar el modelo
    performance = performance_metric(y_test, y_pred_svm)

    # Imprimir los resultados de la m�trica
    print(f"\nFeature Selection Technique: {name}")
    print(f"SVM Model Performance (Precision): {performance}")
    print(f"Selected Features Indices: {selector.get_support(indices=True)}")